# Defining the model and the Image loading function

In [2]:
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, concatenate, Dropout
from tensorflow.keras.models import Model,Sequential
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from warnings import filterwarnings
filterwarnings('ignore')
import tensorflow as tf
import numpy as np

tf.keras.backend.clear_session()

def preprocess_image(image_path,target_size=(224,224)):
    img = load_img(image_path, target_size=target_size)
    img_array = img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array /= 255.0
    return img_array

image_input_shape = (300,300,3)

image_input = Input(shape=image_input_shape, name='image_input')

x = Conv2D(32, (3, 3), activation='relu')(image_input)
x = MaxPooling2D(pool_size=(2, 2))(x)
x = Conv2D(64, (3, 3), activation='relu')(x)
x = MaxPooling2D(pool_size=(2, 2))(x)
x = Conv2D(128, (3, 3), activation='relu')(x)
x = MaxPooling2D(pool_size=(2, 2))(x)
x = Flatten()(x)

# Structured data processing
structured_input = Input(shape=(3,), name='structured_input')  # 3 features: follow-ups, age, gender
y = Dense(32, activation='relu')(structured_input)
y = Dense(16, activation='relu')(y)

# Combine both outputs
combined = concatenate([x, y])

# Add fully connected layers
z = Dense(512, activation='relu')(combined)
z = Dense(2, activation='sigmoid')(z)

# Define the model
model = Model(inputs=[image_input, structured_input], outputs=z)

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Model summary
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ image_input         │ (None, 300, 300,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d (Conv2D)     │ (None, 298, 298,  │        896 │ image_input[0][0] │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d       │ (None, 149, 149,  │          0 │ conv2d[0][0]      │
│ (MaxPooling2D)      │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_1 (Conv2D)   │ (None, 147, 147,  │     18,496 │ max_pooling2d[0]… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_1     │ (None, 73, 73,    │          0 │ conv2d_1[0][0]    │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_2 (Conv2D)   │ (None, 71, 71,    │     73,856 │ max_pooling2d_1[… │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ structured_input    │ (None, 3)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_2     │ (None, 35, 35,    │          0 │ conv2d_2[0][0]    │
│ (MaxPooling2D)      │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 32)        │        128 │ structured_input… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten (Flatten)   │ (None, 156800)    │          0 │ max_pooling2d_2[… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (None, 16)        │        528 │ dense[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, 156816)    │          0 │ flatten[0][0],    │
│ (Concatenate)       │                   │            │ dense_1[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_2 (Dense)     │ (None, 512)       │ 80,290,304 │ concatenate[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_3 (Dense)     │ (None, 2)         │      1,026 │ dense_2[0][0]     │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 80,385,234 (306.65 MB)

 Trainable params: 80,385,234 (306.65 MB)

 Non-trainable params: 0 (0.00 B)

# Importing the data training data set

In [3]:
import pandas as pd

In [4]:
train = pd.read_csv('train_set.csv')

# Encoding Categorical Data 

In [5]:
train['Patient Gender'].replace({'M':0,'F':1},inplace=True)

In [6]:
train['View Position'].replace({'AP':0,'PA':1},inplace=True)

In [7]:
train.head()

,Image Index,Finding Labels,Follow-up,Patient ID,Patient Age,Patient Gender,View Position,OriginalImage_Width,OriginalImage_Height,OriginalImagePixelSpacing_x,OriginalImagePixelSpacing_y
0,00000011_000.png,0,0,11,75,0,1,2638,2449,0.143,0.143
1,00000013_011.png,1,11,13,60,0,0,3056,2544,0.139,0.139
2,00000013_012.png,1,12,13,60,0,0,3056,2544,0.139,0.139
3,00000013_013.png,1,13,13,60,0,0,3056,2544,0.139,0.139
4,00000013_036.png,1,36,13,56,0,1,2952,2968,0.143,0.143


# Image Preprocessing

In [8]:
image_paths = train.apply(lambda row: '/'+str(row['Finding Labels'])+'/'+row['Image Index'],axis=1)

In [9]:
import numpy as np
images = []

for path in image_paths:
    images.append(preprocess_image('train_images'+path,target_size=image_input_shape[0:2]))

x_ray_train = np.array(images)

In [10]:
x_ray_train = x_ray_train.reshape(-1, image_input_shape[0], image_input_shape[1], image_input_shape[2])

In [11]:
x_ray_train.shape

(4305, 300, 300, 3)

In [12]:
labels = pd.get_dummies(train['Finding Labels'])

In [13]:
labels = np.int64(labels)

In [14]:
labels

array([[1, 0],
       [0, 1],
       [0, 1],
       ...,
       [1, 0],
       [1, 0],
       [0, 1]])

In [15]:
structured_data = train[['Follow-up','Patient Age','Patient Gender']]

In [16]:
test=pd.read_csv('test_set.csv')

In [17]:
test['Patient Gender'].replace({'M':0,'F':1},inplace=True)

In [18]:
test['View Position'].replace({'AP':0,'PA':1},inplace=True)

In [19]:
structured_data_test = test[['Follow-up','Patient Age','Patient Gender']]

In [20]:
image_paths_test = test.apply(lambda row: '/test_images/'+row['Image Index'],axis=1)

In [21]:
import numpy as np
images = []

for path in image_paths_test:
    images.append(preprocess_image('test_images'+path,target_size=image_input_shape[0:2]))

x_ray_test = np.array(images)

In [22]:
x_ray_test = x_ray_test.reshape(-1, image_input_shape[0], image_input_shape[1], image_input_shape[2])

In [23]:
x_ray_test.shape

(1844, 300, 300, 3)

### Training - 5 Fold Cross Validation

In [25]:
from tensorflow.keras.callbacks import ModelCheckpoint,EarlyStopping
from tensorflow.keras.callbacks import ReduceLROnPlateau

reduce_lr = ReduceLROnPlateau(
    monitor='val_accuracy',  
    factor=0.1,          
    patience=10,             
    cooldown=0,          
    min_lr=0.00001       
)

early_stop =EarlyStopping(monitor='val_loss',patience=2,restore_best_weights=True)

In [26]:
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score

k=5
kf = KFold(n_splits=k, shuffle=True, random_state=42)

accuracy_scores = []
model_names = []
i=1
for train_index, val_index in kf.split(x_ray_train):
    model_names.append(f'best_model_300_{i}.keras')

    x_train, x_val = x_ray_train[train_index], x_ray_train[val_index]
    structured_train, structured_val = structured_data.iloc[train_index], structured_data.iloc[val_index]
    labels_train, labels_val = labels[train_index], labels[val_index]
    
    checkpoint = ModelCheckpoint(f'best_model_300_{i}.keras',monitor = 'val_accuracy',save_best_only=True,mode='max')
    
    model.fit(
        [x_train,structured_train],
        labels_train,# Labels
        epochs=6,
        validation_data=[[x_val,structured_val],labels_val],
        batch_size=16,
        callbacks= [reduce_lr,early_stop,checkpoint] 
    )

    model = tf.keras.models.load_model(f'best_model_300_{i}.keras')
    labels_pred = model.predict([x_val,structured_val])
    y_pred = [np.argmax(i) for i in labels_pred]
    y_val = [np.argmax(i) for i in labels_val]
    accuracy_scores.append(accuracy_score(y_val,y_pred))

    print(f"Iteration {i}: Done\n\n")
    i+=1

Epoch 1/6
216/216 ━━━━━━━━━━━━━━━━━━━━ 484s 2s/step - accuracy: 0.6346 - loss: 0.8951 - val_accuracy: 0.6365 - val_loss: 0.6646 - learning_rate: 0.0010
Epoch 2/6
216/216 ━━━━━━━━━━━━━━━━━━━━ 471s 2s/step - accuracy: 0.6504 - loss: 0.6240 - val_accuracy: 0.6423 - val_loss: 0.6407 - learning_rate: 0.0010
Epoch 3/6
216/216 ━━━━━━━━━━━━━━━━━━━━ 456s 2s/step - accuracy: 0.6840 - loss: 0.5930 - val_accuracy: 0.6806 - val_loss: 0.6327 - learning_rate: 0.0010
Epoch 4/6
216/216 ━━━━━━━━━━━━━━━━━━━━ 448s 2s/step - accuracy: 0.7184 - loss: 0.5395 - val_accuracy: 0.6434 - val_loss: 0.6911 - learning_rate: 0.0010
Epoch 5/6
216/216 ━━━━━━━━━━━━━━━━━━━━ 464s 2s/step - accuracy: 0.7581 - loss: 0.4703 - val_accuracy: 0.6876 - val_loss: 0.7351 - learning_rate: 0.0010
27/27 ━━━━━━━━━━━━━━━━━━━━ 21s 765ms/step
Iteration 1: Done


Epoch 1/6
216/216 ━━━━━━━━━━━━━━━━━━━━ 453s 2s/step - accuracy: 0.7922 - loss: 0.4493 - val_accuracy: 0.8014 - val_loss: 0.4329 - learning_rate: 0.0010
Epoch 2/6
216/216 ━━━━━━━━

## Testing

In [27]:
for model_name in model_names:
    model = tf.keras.models.load_model(model_name)
    y_pred = model.predict([x_ray_train,structured_data])
    res = [np.argmax(i) for i in y_pred]
    print(f'Model Name: {model_name}')
    print(f'Accuracy Score: {accuracy_score(train['Finding Labels'],res)}\n')

135/135 ━━━━━━━━━━━━━━━━━━━━ 87s 636ms/step
Model Name: best_model_300_1.keras
Accuracy Score: 0.7718931475029036

135/135 ━━━━━━━━━━━━━━━━━━━━ 91s 668ms/step
Model Name: best_model_300_2.keras
Accuracy Score: 0.8845528455284553

135/135 ━━━━━━━━━━━━━━━━━━━━ 83s 614ms/step
Model Name: best_model_300_3.keras
Accuracy Score: 0.9272938443670151

135/135 ━━━━━━━━━━━━━━━━━━━━ 85s 626ms/step
Model Name: best_model_300_4.keras
Accuracy Score: 0.9291521486643438

135/135 ━━━━━━━━━━━━━━━━━━━━ 90s 663ms/step
Model Name: best_model_300_5.keras
Accuracy Score: 0.956562137049942



In [28]:
import tensorflow as tf

In [74]:
best_model = tf.keras.models.load_model('best_model_300_1.keras')

In [75]:
y_pred = best_model.predict([x_ray_test,structured_data_test])

58/58 ━━━━━━━━━━━━━━━━━━━━ 37s 638ms/step


In [76]:
res = [np.argmax(i) for i in y_pred]

In [77]:
result = pd.DataFrame()

In [78]:
result['Image Index'] = test['Image Index']

In [79]:
result['Finding Labels'] = res

In [80]:
result.head()

,Image Index,Finding Labels
0,00000013_020.png,0
1,00000013_021.png,0
2,00000013_033.png,1
3,00000025_000.png,0
4,00000061_002.png,0


In [81]:
result['Finding Labels'].value_counts(normalize=True)

Finding Labels
0    0.853579
1    0.146421
Name: proportion, dtype: float64

In [82]:
result.to_csv('submission_v27.csv',index=False)